In [104]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [105]:
# only run this once ever per session
# try:
#     whole = pd.read_csv("signs.csv")

# except:
#     print("No dataset found")

In [106]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [107]:
cap = cv2.VideoCapture(0)

ls = []
with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                ls.append([[lmk.x, lmk.y] for lmk in hand_landmarks.landmark])

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [108]:
# normalize coordinates to be relative centered at landmark 0
ls = np.array(ls)

for l in ls:
    for i in range(21):
        l[i][0] = l[i][0] - l[0][0]
        l[i][1] = l[i][1] - l[0][1]

ls

array([[[0.        , 0.        ],
        [0.43936503, 0.82883239],
        [0.47370803, 0.80524611],
        ...,
        [0.41144747, 0.69201577],
        [0.43396145, 0.69289559],
        [0.44878817, 0.70648545]],

       [[0.        , 0.        ],
        [0.43625173, 0.83216006],
        [0.47266185, 0.80426019],
        ...,
        [0.40938911, 0.69327873],
        [0.4333992 , 0.69432598],
        [0.44820988, 0.70900691]],

       [[0.        , 0.        ],
        [0.43648052, 0.83321702],
        [0.47388488, 0.80534774],
        ...,
        [0.41070062, 0.68811047],
        [0.43496194, 0.69194508],
        [0.44990349, 0.71053588]],

       ...,

       [[0.        , 0.        ],
        [0.42554864, 0.7270484 ],
        [0.46000299, 0.71205157],
        ...,
        [0.42556474, 0.5838092 ],
        [0.4449338 , 0.58564758],
        [0.45561346, 0.60274297]],

       [[0.        , 0.        ],
        [0.44829559, 0.61614323],
        [0.46606603, 0.59842378],
        .

In [109]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 42)
reshaped_ls.shape

(1305, 42)

In [110]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")

In [111]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.shape

(1305, 40)

In [112]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "c"

In [113]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(3644, 42)

In [122]:
whole["Label"].value_counts()

c           1305
okay         664
b            642
a            457
good         329
not good     247
Name: Label, dtype: int64

In [123]:
whole.shape

(3644, 42)

In [55]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "c")

(1240, 42)

In [124]:
whole.to_csv("signs_en.csv")